In [1]:
! rm -rf metastore_db/

In [2]:
spark.stop()

In [3]:
spark = SparkSession \
    .builder \
    .appName("Prediction of Loan Payment") \
    .master("spark://192.168.10.1:7077") \
    .config("spark.executor.cores","1")\
    .getOrCreate();

# 1. Data Understanding

## Collecting the current data from the LendingClub's Website.

In [4]:
#! rm -rf LoanStats_web.csv

In [5]:
#! touch LoanStats_web.csv

In [6]:
! ls -l /root/notebook/data/LoanStats_web.csv

-rw-r--r-- 1 root root 1160243241 Nov 22  2019 /root/notebook/data/LoanStats_web.csv


In [7]:
! head -3 /root/notebook/data/LoanStats_web.csv

"id","member_id","loan_amnt","funded_amnt","funded_amnt_inv","term","int_rate","installment","grade","sub_grade","emp_title","emp_length","home_ownership","annual_inc","verification_status","issue_d","loan_status","pymnt_plan","url","desc","purpose","title","zip_code","addr_state","dti","delinq_2yrs","earliest_cr_line","inq_last_6mths","mths_since_last_delinq","mths_since_last_record","open_acc","pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv","total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","total_rec_late_fee","recoveries","collection_recovery_fee","last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med","mths_since_last_major_derog","policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq","tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","open_il_12m","open_il_24m","mths_since_rcnt_il","total_bal_il","il_util","open_rv_1

In [8]:
! wc -l /root/notebook/data/LoanStats_web.csv

1432493 /root/notebook/data/LoanStats_web.csv


In [9]:
raw_df = spark.read.format('csv').option('header','true').option('mode','DROPMALFORMED')\
.load('/root/notebook/data/LoanStats_web.csv')

In [10]:
raw201617_df = raw_df

In [11]:
raw201617_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: string (nullable = true)
 |-- funded_amnt: string (nullable = true)
 |-- funded_amnt_inv: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

### Numbers of Fields [1]

In [12]:
len(raw201617_df.columns)

144

In [13]:
rawweb_df = raw201617_df.select('id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 'dti_joint',
 'verification_status_joint',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'open_acc_6m',
 #'open_il_6m',
 'open_il_12m',
 'open_il_24m',
 'mths_since_rcnt_il',
 'total_bal_il',
 'il_util',
 'open_rv_12m',
 'open_rv_24m',
 'max_bal_bc',
 'all_util',
 'total_rev_hi_lim',
 'inq_fi',
 'total_cu_tl',
 'inq_last_12m')

### Numbers of Fields after attributes selections [1]

In [14]:
len(rawweb_df.columns)

73

In [15]:
rawweb_df.select(['issue_d']).distinct().show()

+--------+
| issue_d|
+--------+
|Oct-2016|
|Feb-2019|
|Mar-2018|
|Sep-2018|
|Jan-2016|
|Jul-2016|
|Aug-2016|
|Feb-2017|
|Nov-2018|
|Jul-2018|
|Apr-2018|
|Dec-2018|
|May-2016|
|Jan-2018|
|Aug-2018|
|Jan-2019|
|Apr-2016|
|Sep-2016|
|Oct-2018|
|Feb-2018|
+--------+
only showing top 20 rows



In [16]:
raw_df = rawweb_df

In [17]:
raw_df.count()

1432439

In [18]:
df_colfam5 = raw_df.select("loan_amnt","term","int_rate","installment","grade","emp_length",\
                           "home_ownership","annual_inc","verification_status","loan_status",\
                           "purpose","addr_state","dti","delinq_2yrs","earliest_cr_line",\
                           "inq_last_6mths","open_acc","pub_rec","revol_bal","revol_util","total_acc",\
                           "last_credit_pull_d")

### Numbers of Fields after final attributes selections [1]

In [19]:
len(df_colfam5.columns)

22

In [20]:
! pip install pandas

You are using pip version 9.0.1, however version 20.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [21]:
import pandas as pd
pd.DataFrame(df_colfam5.describe().take(6), columns=df_colfam5.describe().columns).transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
loan_amnt,1432439,15370.392945179516,9646.028077240424,1000,9975
term,1432439,None,None,36 months,60 months
int_rate,1432439,None,None,5.31%,30.99%
installment,1432439,456.6879090138958,281.71351826414445,100.01,999.98
grade,1432439,None,None,A,G
emp_length,1432439,None,None,1 year,n/a
home_ownership,1432439,None,None,ANY,RENT
annual_inc,1432439,81034.5941268703,134183.40313571738,0,99999.84
verification_status,1432439,None,None,Not Verified,Verified


df_colfam5.select(['last_credit_pull_d']).distinct().\
orderBy(['last_credit_pull_d'],ascending=1).show()

df_colfam5.select(['last_credit_pull_d']).\
filter(df_colfam5.last_credit_pull_d == '').count()

df_colfam5.select(['last_credit_pull_d']).\
filter(df_colfam5['last_credit_pull_d'].isNull()).count()

In [22]:
df_colfam5.groupby('home_ownership').count().show()

+--------------+------+
|home_ownership| count|
+--------------+------+
|           OWN|170888|
|          RENT|556960|
|      MORTGAGE|701646|
|           ANY|  2940|
|          NONE|     5|
+--------------+------+



# 2. Data Preparation

Divide this process into 2 parts. The first is a business oriented preparation that turn many business rules to be programming's logics. Its result benefits many tasks related to the Business Intelligence and other descriptive analytics. The second one is a data science oriented preparation that turn many requirements of data science to be programming's logics. Its result benefits many tasks related to the Predictive analytics.

Asumption: Business oriented preparation: No missing values, ONLY month, Correct data types.

## Data Cleansing: Remove missing values

In [23]:
df_no_missing = df_colfam5.dropna(how='any')

## Check for a schema

In [24]:
df_no_missing.printSchema()

root
 |-- loan_amnt: string (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: string (nullable = true)
 |-- installment: string (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: string (nullable = true)
 |-- pub_rec: string (nullable = true)
 |-- revol_bal: string (nullable = true)
 |-- revol_util: string (nullable = true)
 |-- total_acc: string (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



In [25]:
df_no_missing_fitmem = df_no_missing.repartition(60)

In [26]:
df_no_missing_cached = df_no_missing_fitmem.cache()

In [27]:
df_no_missing_cached.registerTempTable("df")

## Number of data rows

In [28]:
df_no_missing_cached.count()

1429645

## Data Transformation: Remove Sign of Percent and Extract Month.

In [29]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

### Remove Sign of Percent

In [30]:
def f_removepercentsign(origin):
    return origin.rstrip('%')

In [31]:
removepercentsign = udf(lambda x: f_removepercentsign(x),StringType())

### Extract Month

In [32]:
def f_extractmonth(origin):
    return origin.split('-')[0]

In [33]:
extractmonth = udf(lambda x: f_extractmonth(x),StringType())

In [34]:
extractterm = udf(lambda x: x.replace('months',''),StringType())

In [35]:
#extractemp_length = udf(lambda x: x.replace('year',''),StringType())

In [36]:
from pyspark.sql.functions import col

In [37]:
dfWithCrunch = df_no_missing_cached.\
withColumn('revol_util',removepercentsign(col('revol_util')).cast(DoubleType())).\
withColumn('int_rate',removepercentsign(col('int_rate')).cast(DoubleType())).\
withColumn('earliest_cr_line',extractmonth(col('earliest_cr_line')).cast(StringType())).\
withColumn('last_credit_pull_d',extractmonth(col('last_credit_pull_d')).cast(StringType())).\
withColumn('dti',col('dti').cast(DoubleType())).\
withColumn('loan_amnt',col('loan_amnt').cast(DoubleType())).\
withColumn('revol_bal',col('revol_bal').cast(DoubleType())).\
withColumn('term',extractterm(col('term')).cast(DoubleType())).\
withColumn('installment',col('installment').cast(DoubleType())).\
withColumn('open_acc',col('open_acc').cast(DoubleType())).\
withColumn('total_acc',col('total_acc').cast(DoubleType())).\
withColumn('pub_rec',col('pub_rec').cast(DoubleType())).\
withColumn('annual_inc',col('annual_inc').cast(DoubleType()))

In [38]:
#dfWithCrunch.select(['last_credit_pull_d']).printSchema()

In [39]:
#dfWithCrunch.groupby('emp_length').count().show()

In [40]:
#dfWithCrunch.count()

In [41]:
dfWithCrunch.select(['loan_amnt','int_rate','dti','home_ownership','grade','term']).show()

+---------+--------+-----+--------------+-----+----+
|loan_amnt|int_rate|  dti|home_ownership|grade|term|
+---------+--------+-----+--------------+-----+----+
|   7000.0|    7.99|23.64|          RENT|    A|36.0|
|  20000.0|   12.74|24.29|          RENT|    C|60.0|
|   5000.0|   11.49|22.63|      MORTGAGE|    B|36.0|
|   7500.0|   11.44| 7.08|          RENT|    B|36.0|
|   6000.0|   11.49| 18.8|          RENT|    B|36.0|
|  12425.0|   14.99| 13.7|      MORTGAGE|    C|36.0|
|   6200.0|    6.99| 8.71|      MORTGAGE|    A|36.0|
|  30000.0|   13.99|30.88|      MORTGAGE|    C|36.0|
|  20000.0|    8.24|10.53|          RENT|    B|36.0|
|   7200.0|    7.49|33.01|      MORTGAGE|    A|36.0|
|   5000.0|   12.74|34.12|           OWN|    C|36.0|
|  20000.0|   13.99|17.61|      MORTGAGE|    C|36.0|
|  33000.0|    6.99|16.24|      MORTGAGE|    A|36.0|
|  12000.0|   12.74|17.24|      MORTGAGE|    C|36.0|
|  40000.0|   10.49| 3.63|      MORTGAGE|    B|36.0|
|   8400.0|    7.24|24.38|      MORTGAGE|    A

In [42]:
#dfWithCrunch.select(['loan_amnt','int_rate','dti','home_ownership','grade']).printSchema()

In [43]:
dfWithCrunch.select(['loan_amnt','annual_inc']).describe().show()

+-------+------------------+-----------------+
|summary|         loan_amnt|       annual_inc|
+-------+------------------+-----------------+
|  count|           1429645|          1429645|
|   mean|15366.699302973815| 81111.6634481217|
| stddev|  9641.85136599622|134222.5400492037|
|    min|            1000.0|              0.0|
|    max|           40000.0|            6.1E7|
+-------+------------------+-----------------+



In [44]:
rawhive_df = dfWithCrunch.repartition(60).cache()
#rawhive_df.registerTempTable("crunched_data")

In [45]:
len(rawhive_df.columns)

22

In [46]:
rawhive_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: double (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



In [47]:
#rawhive_df.select(['loan_amnt','int_rate','dti','home_ownership','grade']).printSchema()

In [48]:
rawhive_df.count()

1429645

### Data Transformation: Normalization of "annual_inc" and "loan_amnt"

In [49]:
from pyspark.sql.functions import *

max_annual_inc = rawhive_df.select(max('annual_inc')).collect()[0][0]

min_annual_inc = rawhive_df.select(min('annual_inc')).collect()[0][0]

#sqlContext.udf.register("t_annual_inc", lambda x: ((x-min_annual_inc)/(max_annual_inc-min_annual_inc)))

In [50]:
def t_annual_inc(origin):
    return ((origin-min_annual_inc)/(max_annual_inc-min_annual_inc))

In [51]:
normalized_annual_inc = udf(lambda x: t_annual_inc(x),DoubleType())

In [52]:
max_loan_amnt = rawhive_df.select(max('loan_amnt')).collect()[0][0]

min_loan_amnt = rawhive_df.select(min('loan_amnt')).collect()[0][0]

#sqlContext.udf.register("t_loan_amnt", lambda x: ((x-min_loan_amnt)/(max_loan_amnt-min_loan_amnt)))

In [53]:
def t_loan_amnt(origin):
    return ((origin-min_loan_amnt)/(max_loan_amnt-min_loan_amnt))

In [54]:
normalized_loan_amnt = udf(lambda x: t_loan_amnt(x),DoubleType())

In [55]:
normalized_df = rawhive_df.withColumn('loan_amnt',normalized_loan_amnt(col('loan_amnt'))).\
withColumn('annual_inc',normalized_annual_inc(col('annual_inc')))

In [56]:
normalized_df.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: double (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: string (nullable = true)
 |-- open_acc: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: double (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



### Number of data rows that are only "Fully Paid" and "Charged Off"

In [57]:
from pyspark.sql.functions import col

In [58]:
normalized_df.select(['loan_amnt','term','int_rate','installment','grade','annual_inc','revol_bal','loan_status']).\
orderBy(['loan_amnt','term','int_rate','installment','grade','annual_inc']).show()

+---------+----+--------+-----------+-----+--------------------+---------+-----------+
|loan_amnt|term|int_rate|installment|grade|          annual_inc|revol_bal|loan_status|
+---------+----+--------+-----------+-----+--------------------+---------+-----------+
|      0.0|36.0|    5.31|      30.12|    A|4.098360655737705E-4|   3215.0|    Current|
|      0.0|36.0|    5.31|      30.12|    A|5.081967213114754E-4|  25993.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|5.901639344262295E-4|   3311.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|6.065573770491804E-4|   1605.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|6.557377049180328E-4|   1329.0|    Current|
|      0.0|36.0|    5.31|      30.12|    A|6.557377049180328E-4|   1016.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|7.180327868852459E-4|    804.0|    Current|
|      0.0|36.0|    5.31|      30.12|    A|9.344262295081968E-4|    858.0| Fully Paid|
|      0.0|36.0|    5.31|      30.12|    A|

In [59]:
normalized_filtered_df = normalized_df.filter(col('loan_status') == 'Fully Paid').\
union(normalized_df.filter(col('loan_status') == 'Charged Off'))

In [60]:
data = normalized_filtered_df.repartition(60).cache()

### Drop Null

In [61]:
data_no_missing_df = data.dropna(how='any')

In [62]:
data_no_missing_df.count()

735740

# 3. Data Modeling

In [63]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder,\
VectorIndexer, QuantileDiscretizer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, GBTClassifier, NaiveBayes\
, RandomForestClassifier, DecisionTreeClassifier, MultilayerPerceptronClassifier
from pyspark.ml import Pipeline
from pyspark.ml.clustering import *

In [64]:
data_no_missing_df_fully_paid = data_no_missing_df.filter(col('loan_status') == 'Fully Paid').sample(True, 0.3, 42)

In [65]:
data_no_missing_df_charge_off = data_no_missing_df.filter(col('loan_status') == 'Charged Off')

In [66]:
final_data_no_missing_df = data_no_missing_df_fully_paid.union(data_no_missing_df_charge_off)

In [67]:
training , test = final_data_no_missing_df.filter(col('loan_amnt') > 0).randomSplit([0.8,0.2])

## Calculation for Confusion Matrix.

In [68]:
def eval_metrics(lap):

    tp = float(len(lap[(lap['loan_status']=='Charged Off') &\
                       (lap['prediction_label']=='Charged Off')]))
    
    fn = float(len(lap[(lap['loan_status']=='Charged Off') &\
                       (lap['prediction_label']=='Fully Paid')]))

    tn = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                       (lap['prediction_label']=='Fully Paid')]))

    fp = float(len(lap[(lap['loan_status']=='Fully Paid') &\
                       (lap['prediction_label']=='Charged Off')]))


    try:
        positivepredictivevalue = tp / (tp+fp)
        negativepredictivevalue = tn / (tn+fn)
        sensitivity = tp / (tp+fn)
        specificity = tn / (tn+fp)
        accuracy = (tp+tn) / (tp+tn+fp+fn)
        return {'PPV': positivepredictivevalue, 'Sensitivity': sensitivity,\
            'NPV':negativepredictivevalue, 'Specificity': specificity, 'Accuracy': accuracy}
    except ZeroDivisionError:
        return 0

    

In [69]:
! pip install pandas

You are using pip version 9.0.1, however version 20.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


# Jan2020

In [70]:
from pyspark.ml.feature import QuantileDiscretizer
from pyspark.ml.feature import Bucketizer

In [71]:
def get_model(df,categoricalCols,continuousCols,\
              discretedCols,split_range,labelCol):

    from pyspark.ml import Pipeline
    from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
    from pyspark.sql.functions import col
    
    
    labelIndexer = StringIndexer(inputCol=labelCol,\
                             outputCol='indexedLabel',\
                             handleInvalid='keep')

    indexers = [ StringIndexer(handleInvalid='keep',\
                               inputCol=c, outputCol="{0}_indexed".format(c))
                 for c in categoricalCols ]

    # default setting: dropLast=True
    encoders = [ OneHotEncoder(inputCol=indexer.getOutputCol(),
                 outputCol="{0}_encoded".format(indexer.getOutputCol()))
                 for indexer in indexers ]
    discretizers = [ Bucketizer(inputCol=d, outputCol="{0}_discretized".format(d)\
                 ,splits=split_range)
                 for d in discretedCols ]
    
    
    featureCols = ['features']
    assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders]
                                + continuousCols +\
                                [discretizer.getOutputCol() for discretizer in discretizers], \
                                outputCol='features')
    

    pipeline_feature = Pipeline(stages=[labelIndexer] + indexers + encoders + discretizers + \
                        [assembler])
    
    
    
    train_df_features = pipeline_feature.fit(training).transform(training)
    
    layers = [
    train_df_features.schema["features"].metadata["ml_attr"]["num_attrs"],
    20, 10, 5, 2]
    
    
    clf = MultilayerPerceptronClassifier(labelCol='indexedLabel',featuresCol='features'\
                                     ,layers=layers)
    
    
    pipeline = Pipeline(stages=[labelIndexer] + indexers + encoders + discretizers + \
                        [assembler] + [clf])
    


    model=pipeline.fit(df)


    return model

In [72]:
training

DataFrame[loan_amnt: double, term: double, int_rate: double, installment: double, grade: string, emp_length: string, home_ownership: string, annual_inc: double, verification_status: string, loan_status: string, purpose: string, addr_state: string, dti: double, delinq_2yrs: string, earliest_cr_line: string, inq_last_6mths: string, open_acc: double, pub_rec: double, revol_bal: double, revol_util: double, total_acc: double, last_credit_pull_d: string]

In [73]:
training.select('grade').describe().show()

+-------+------+
|summary| grade|
+-------+------+
|  count|261100|
|   mean|  null|
| stddev|  null|
|    min|     A|
|    max|     G|
+-------+------+



In [74]:
catcols = [\
           'emp_length',\
           'home_ownership',\
           'verification_status',\
           #'purpose',\
           'grade',\
           'addr_state'\
          ]

num_cols = [\
            #'dti',\
            'loan_amnt',\
            'int_rate',\
            'installment',\
            'annual_inc',\
            #'revol_bal',\
            #'delinq_2yrs',\
            'pub_rec',\
            #'revol_util'\
            'total_acc'\
           ]

discols = [\
           #'pub_rec',\
           'dti',\
           #'installment'\
          ]



labelCol = 'loan_status'

splits = [-1.0, 0.0, 1.0, 2.0, 3.0, 4.0, float("inf")]

In [75]:
from itertools import combinations, chain 
list(combinations(num_cols, 2))

[('loan_amnt', 'int_rate'),
 ('loan_amnt', 'installment'),
 ('loan_amnt', 'annual_inc'),
 ('loan_amnt', 'pub_rec'),
 ('loan_amnt', 'total_acc'),
 ('int_rate', 'installment'),
 ('int_rate', 'annual_inc'),
 ('int_rate', 'pub_rec'),
 ('int_rate', 'total_acc'),
 ('installment', 'annual_inc'),
 ('installment', 'pub_rec'),
 ('installment', 'total_acc'),
 ('annual_inc', 'pub_rec'),
 ('annual_inc', 'total_acc'),
 ('pub_rec', 'total_acc')]

In [76]:
for k in range(len(num_cols)):
    for l in (list(combinations(num_cols, k+1))):
        print(list(l))
            

['loan_amnt']
['int_rate']
['installment']
['annual_inc']
['pub_rec']
['total_acc']
['loan_amnt', 'int_rate']
['loan_amnt', 'installment']
['loan_amnt', 'annual_inc']
['loan_amnt', 'pub_rec']
['loan_amnt', 'total_acc']
['int_rate', 'installment']
['int_rate', 'annual_inc']
['int_rate', 'pub_rec']
['int_rate', 'total_acc']
['installment', 'annual_inc']
['installment', 'pub_rec']
['installment', 'total_acc']
['annual_inc', 'pub_rec']
['annual_inc', 'total_acc']
['pub_rec', 'total_acc']
['loan_amnt', 'int_rate', 'installment']
['loan_amnt', 'int_rate', 'annual_inc']
['loan_amnt', 'int_rate', 'pub_rec']
['loan_amnt', 'int_rate', 'total_acc']
['loan_amnt', 'installment', 'annual_inc']
['loan_amnt', 'installment', 'pub_rec']
['loan_amnt', 'installment', 'total_acc']
['loan_amnt', 'annual_inc', 'pub_rec']
['loan_amnt', 'annual_inc', 'total_acc']
['loan_amnt', 'pub_rec', 'total_acc']
['int_rate', 'installment', 'annual_inc']
['int_rate', 'installment', 'pub_rec']
['int_rate', 'installment', 't

In [77]:
spark_labeling = udf(lambda x: "Charged Off" if x == 1.0 else "Fully Paid")

In [78]:
result_list = []

In [79]:
for i in range(len(catcols)):
    for j in (list(combinations(catcols, i+1))):
        for k in range(len(num_cols)):
            for l in (list(combinations(num_cols, k+1))):
                l_list = list(l)
                ml_model = get_model(training,j,l_list,discols, splits, labelCol)
                lap2 = ml_model.transform(test)\
                .withColumn('prediction_label',spark_labeling(col('prediction')))\
                .toPandas()
                m2 = eval_metrics(lap2)
                result_text = "รอบแรกใช้ categorical {0} และ numerical {1} ได้ Model ที่มี {2}".format(j,l_list,m2)
                print(result_text)
                result_list.append(result_text)
                
        

รอบแรกใช้ categorical ('emp_length',) และ numerical ['loan_amnt'] ได้ Model ที่มี {'PPV': 0.5352261838721065, 'Sensitivity': 0.3501715718622146, 'NPV': 0.5635263612791702, 'Specificity': 0.7339799099411154, 'Accuracy': 0.5548866855524079}
รอบแรกใช้ categorical ('emp_length',) และ numerical ['int_rate'] ได้ Model ที่มี {'PPV': 0.6171826942792701, 'Sensitivity': 0.5958822753068497, 'NPV': 0.6568226393947885, 'Specificity': 0.676653966054728, 'Accuracy': 0.6389641581475551}
รอบแรกใช้ categorical ('emp_length',) และ numerical ['installment'] ได้ Model ที่มี 0
รอบแรกใช้ categorical ('emp_length',) และ numerical ['annual_inc'] ได้ Model ที่มี {'PPV': 0.573597232897771, 'Sensitivity': 0.09848884782895605, 'NPV': 0.5426959898239272, 'Specificity': 0.9359485047915945, 'Accuracy': 0.5451718191895554}
รอบแรกใช้ categorical ('emp_length',) และ numerical ['pub_rec'] ได้ Model ที่มี {'PPV': 0.541671090349294, 'Sensitivity': 0.1683383925036294, 'NPV': 0.5461077197342121, 'Specificity': 0.875389677866

รอบแรกใช้ categorical ('emp_length',) และ numerical ['int_rate', 'annual_inc', 'total_acc'] ได้ Model ที่มี {'PPV': 0.612692704691506, 'Sensitivity': 0.6071334301174607, 'NPV': 0.6590108536899683, 'Specificity': 0.6642420043874842, 'Accuracy': 0.6375939155068359}
รอบแรกใช้ categorical ('emp_length',) และ numerical ['int_rate', 'pub_rec', 'total_acc'] ได้ Model ที่มี {'PPV': 0.6161051259177932, 'Sensitivity': 0.5924838326514451, 'NPV': 0.6550578115399653, 'Specificity': 0.6770292114074588, 'Accuracy': 0.6375785195221086}
รอบแรกใช้ categorical ('emp_length',) และ numerical ['installment', 'annual_inc', 'pub_rec'] ได้ Model ที่มี 0
รอบแรกใช้ categorical ('emp_length',) และ numerical ['installment', 'annual_inc', 'total_acc'] ได้ Model ที่มี {'PPV': 0.5656771799628942, 'Sensitivity': 0.10060050151775109, 'NPV': 0.5423424330949262, 'Specificity': 0.9324269714813532, 'Accuracy': 0.5442788520753787}
รอบแรกใช้ categorical ('emp_length',) และ numerical ['installment', 'pub_rec', 'total_acc'] ได

รอบแรกใช้ categorical ('home_ownership',) และ numerical ['loan_amnt', 'installment'] ได้ Model ที่มี {'PPV': 0.5471006588420089, 'Sensitivity': 0.41371915005939025, 'NPV': 0.5772607237170795, 'Specificity': 0.7003810183581572, 'Accuracy': 0.5666184259145215}
รอบแรกใช้ categorical ('home_ownership',) และ numerical ['loan_amnt', 'annual_inc'] ได้ Model ที่มี {'PPV': 0.5578003949567066, 'Sensitivity': 0.36346839118384583, 'NPV': 0.5732141672012919, 'Specificity': 0.7479217180464149, 'Accuracy': 0.5685275280206922}
รอบแรกใช้ categorical ('home_ownership',) และ numerical ['loan_amnt', 'pub_rec'] ได้ Model ที่มี {'PPV': 0.5580894587452285, 'Sensitivity': 0.37627029167216575, 'NPV': 0.5753627746080238, 'Specificity': 0.7393488049878767, 'Accuracy': 0.5699285626308659}
รอบแรกใช้ categorical ('home_ownership',) และ numerical ['loan_amnt', 'total_acc'] ได้ Model ที่มี {'PPV': 0.5088875007098643, 'Sensitivity': 0.5913290220403854, 'NPV': 0.5834398331875967, 'Specificity': 0.5007504907054613, 'Acc

รอบแรกใช้ categorical ('home_ownership',) และ numerical ['annual_inc', 'pub_rec', 'total_acc'] ได้ Model ที่มี {'PPV': 0.5206130703185979, 'Sensitivity': 0.4841625973340372, 'NPV': 0.5747701680900832, 'Specificity': 0.6099757533772082, 'Accuracy': 0.5512686291415199}
รอบแรกใช้ categorical ('home_ownership',) และ numerical ['loan_amnt', 'int_rate', 'installment', 'annual_inc'] ได้ Model ที่มี {'PPV': 0.59276596981944, 'Sensitivity': 0.6726606836478818, 'NPV': 0.6753493242579928, 'Specificity': 0.5957164299734442, 'Accuracy': 0.6316202734326888}
รอบแรกใช้ categorical ('home_ownership',) และ numerical ['loan_amnt', 'int_rate', 'installment', 'pub_rec'] ได้ Model ที่มี 0
รอบแรกใช้ categorical ('home_ownership',) และ numerical ['loan_amnt', 'int_rate', 'installment', 'total_acc'] ได้ Model ที่มี {'PPV': 0.6220134995178743, 'Sensitivity': 0.574666754652237, 'NPV': 0.6511325809856296, 'Specificity': 0.6944925528229996, 'Accuracy': 0.6385792585293756}
รอบแรกใช้ categorical ('home_ownership',) 

รอบแรกใช้ categorical ('verification_status',) และ numerical ['loan_amnt', 'pub_rec'] ได้ Model ที่มี {'PPV': 0.552230522945571, 'Sensitivity': 0.42681800184769697, 'NPV': 0.5816697570255497, 'Specificity': 0.6972347304006465, 'Accuracy': 0.5710524695159502}
รอบแรกใช้ categorical ('verification_status',) และ numerical ['loan_amnt', 'total_acc'] ได้ Model ที่มี {'PPV': 0.5437412205400344, 'Sensitivity': 0.45977959614623204, 'NPV': 0.5836384904892686, 'Specificity': 0.6624812377323634, 'Accuracy': 0.5678962926468777}
รอบแรกใช้ categorical ('verification_status',) และ numerical ['int_rate', 'installment'] ได้ Model ที่มี {'PPV': 0.6107906122984136, 'Sensitivity': 0.6148211693282302, 'NPV': 0.6610730461038207, 'Specificity': 0.6572566678212678, 'Accuracy': 0.6374553516442911}
รอบแรกใช้ categorical ('verification_status',) และ numerical ['int_rate', 'annual_inc'] ได้ Model ที่มี {'PPV': 0.6138054050415643, 'Sensitivity': 0.6017553121288108, 'NPV': 0.6574817673600273, 'Specificity': 0.668773

...

รอบแรกใช้ categorical ('emp_length', 'verification_status', 'addr_state') และ numerical ['loan_amnt', 'installment', 'annual_inc', 'pub_rec', 'total_acc'] ได้ Model ที่มี {'PPV': 0.548361858190709, 'Sensitivity': 0.3700013197835555, 'NPV': 0.5709406318817132, 'Specificity': 0.7334026093984528, 'Accuracy': 0.5638317526789013}
รอบแรกใช้ categorical ('emp_length', 'verification_status', 'addr_state') และ numerical ['int_rate', 'installment', 'annual_inc', 'pub_rec', 'total_acc'] ได้ Model ที่มี {'PPV': 0.5640580716062236, 'Sensitivity': 0.6435264616602877, 'NPV': 0.6443010469480477, 'Specificity': 0.5648885809952662, 'Accuracy': 0.6015827072299544}
รอบแรกใช้ categorical ('emp_length', 'verification_status', 'addr_state') และ numerical ['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'pub_rec', 'total_acc'] ได้ Model ที่มี {'PPV': 0.6094641568833213, 'Sensitivity': 0.6132044344727464, 'NPV': 0.6597887283069244, 'Specificity': 0.6562463918716084, 'Accuracy': 0.6361620889272078}
รอบแรก

รอบแรกใช้ categorical ('emp_length', 'grade', 'addr_state') และ numerical ['loan_amnt', 'installment', 'total_acc'] ได้ Model ที่มี {'PPV': 0.585477522825243, 'Sensitivity': 0.721492675201267, 'NPV': 0.6941999058073398, 'Specificity': 0.5531116499249509, 'Accuracy': 0.6316818573715974}
รอบแรกใช้ categorical ('emp_length', 'grade', 'addr_state') และ numerical ['loan_amnt', 'annual_inc', 'pub_rec'] ได้ Model ที่มี {'PPV': 0.6199176618692797, 'Sensitivity': 0.6011614095288372, 'NPV': 0.6600770507016113, 'Specificity': 0.677548781895855, 'Accuracy': 0.6419047912304471}
รอบแรกใช้ categorical ('emp_length', 'grade', 'addr_state') และ numerical ['loan_amnt', 'annual_inc', 'total_acc'] ได้ Model ที่มี {'PPV': 0.6175887001849673, 'Sensitivity': 0.6059126303286261, 'NPV': 0.6608456143339865, 'Specificity': 0.6717757764692299, 'Accuracy': 0.6410426160857249}
รอบแรกใช้ categorical ('emp_length', 'grade', 'addr_state') และ numerical ['loan_amnt', 'pub_rec', 'total_acc'] ได้ Model ที่มี {'PPV': 0.61

รอบแรกใช้ categorical ('emp_length', 'grade', 'addr_state') และ numerical ['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'pub_rec'] ได้ Model ที่มี {'PPV': 0.6092355575316798, 'Sensitivity': 0.6329351986274251, 'NPV': 0.667563125653668, 'Specificity': 0.6448447061540238, 'Accuracy': 0.6392874738268259}
รอบแรกใช้ categorical ('emp_length', 'grade', 'addr_state') และ numerical ['loan_amnt', 'int_rate', 'installment', 'annual_inc', 'total_acc'] ได้ Model ที่มี {'PPV': 0.5987592372958672, 'Sensitivity': 0.6496304606044608, 'NPV': 0.6688702485266145, 'Specificity': 0.6191548320055421, 'Accuracy': 0.6333754156915876}
รอบแรกใช้ categorical ('emp_length', 'grade', 'addr_state') และ numerical ['loan_amnt', 'int_rate', 'installment', 'pub_rec', 'total_acc'] ได้ Model ที่มี {'PPV': 0.5879253321374517, 'Sensitivity': 0.6920944965025736, 'NPV': 0.6812188289950126, 'Specificity': 0.5756263710887888, 'Accuracy': 0.6299729030668801}
รอบแรกใช้ categorical ('emp_length', 'grade', 'addr_state') แ

รอบแรกใช้ categorical ('home_ownership', 'verification_status', 'grade') และ numerical ['loan_amnt', 'int_rate', 'annual_inc'] ได้ Model ที่มี {'PPV': 0.6227138845126379, 'Sensitivity': 0.5999076151511152, 'NPV': 0.6608491357610338, 'Specificity': 0.6820228611014895, 'Accuracy': 0.6437061214435276}
รอบแรกใช้ categorical ('home_ownership', 'verification_status', 'grade') และ numerical ['loan_amnt', 'int_rate', 'pub_rec'] ได้ Model ที่มี {'PPV': 0.6229002980222162, 'Sensitivity': 0.6068694734063613, 'NPV': 0.6636461156278229, 'Specificity': 0.6785879228726475, 'Accuracy': 0.6451225520384284}
รอบแรกใช้ categorical ('home_ownership', 'verification_status', 'grade') และ numerical ['loan_amnt', 'int_rate', 'total_acc'] ได้ Model ที่มี {'PPV': 0.6193327345906928, 'Sensitivity': 0.6143262504949188, 'NPV': 0.6649660351400155, 'Specificity': 0.6696686294885117, 'Accuracy': 0.6438446853060722}
รอบแรกใช้ categorical ('home_ownership', 'verification_status', 'grade') และ numerical ['loan_amnt', 'in

Py4JJavaError: An error occurred while calling o341357.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 33 in stage 908923.0 failed 4 times, most recent failure: Lost task 33.3 in stage 908923.0 (TID 24601048, 192.168.10.4, executor 7): java.io.IOException: No space left on device
	at java.io.FileOutputStream.writeBytes(Native Method)
	at java.io.FileOutputStream.write(FileOutputStream.java:326)
	at org.apache.spark.storage.TimeTrackingOutputStream.write(TimeTrackingOutputStream.java:58)
	at java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:82)
	at java.io.BufferedOutputStream.flush(BufferedOutputStream.java:140)
	at net.jpountz.lz4.LZ4BlockOutputStream.finish(LZ4BlockOutputStream.java:243)
	at net.jpountz.lz4.LZ4BlockOutputStream.close(LZ4BlockOutputStream.java:175)
	at java.io.ObjectOutputStream$BlockDataOutputStream.close(ObjectOutputStream.java:1828)
	at java.io.ObjectOutputStream.close(ObjectOutputStream.java:742)
	at org.apache.spark

In [ ]:
result_list

In [ ]:
result_list